In [8]:
import sys
import os

import whisper
import pysrt

In [8]:
#TODO try another video-try loaded model
model = whisper.load_model("medium")

In [1]:
import joblib
model = joblib.load("C:\dev\whisper_models\whisper_small.pkl") 

In [2]:

result = model.transcribe("C:\\Users\\vlns\\Desktop\\sub_ex2\\My Hero Academia S01E10 MULTi 1080p 10bits BluRay x265 AAC -Punisher694.mkv")

In [13]:

result2 = model.transcribe("C:\\Users\\vlns\Desktop\\sub_ex\\My Hero Academia S01E07 MULTi 1080p 10bits BluRay x265 AAC -Punisher694.mkv")

In [6]:
def convert_time(seconds):

    hours = int(seconds // 3600)
    remaining_seconds = seconds % 3600
    
    minutes = int(remaining_seconds // 60)
    remaining_seconds %= 60
    
    seconds = int(remaining_seconds)
    miliseconds = int((remaining_seconds - seconds) * 1000)
    
    return (hours, minutes, seconds, miliseconds)


In [5]:

def result_to_srt(result, output_file, plus_time=0):
    #We create our srt object
    subs = pysrt.SubRipFile()

    if not plus_time:
        #If plus-time method isnt used, we create subtitle instances for every sentence
        #in our result, using its start-end timestamps
        for segment in result['segments']:
            start_time = convert_time(segment['start']) 
            end_time = convert_time(segment['end'])
            
            sub_item = pysrt.SubRipItem(start=start_time, 
                                        end=end_time, 
                                        text=segment['text'])
            subs.append(sub_item)

    else:
        #If plus-time is used we set end time of instances as start of the next one
        for i in range(len(result['segments']) - 1):
            start_time = convert_time(result['segments'][i]['start'])
            end_time = convert_time(min(result['segments'][i+1]['start'], 
                                        (result['segments'][i]['end']) + plus_time))
            
            sub_item = pysrt.SubRipItem(start=start_time, 
                                        end=end_time, 
                                        text=result['segments'][i]['text'])
            subs.append(sub_item)
        last_sub_item = pysrt.SubRipItem(start=convert_time(result['segments'][-1]['start']), 
                                        end=convert_time(result['segments'][-1]['end']), 
                                        text=result['segments'][-1]['text'])
    subs.save(output_file, encoding='utf-8')
    print(f'Subtitle file {output_file} is ready.')   



In [1]:
result_to_srt(result, "C:\\Users\\vlns\\Desktop\\sub_ex2\\My Hero Academia S01E10 MULTi 1080p 10bits BluRay x265 AAC -Punisher69412.srt")

NameError: name 'result_to_srt' is not defined

In [ ]:
def subtitles_for_list(model, video_list, sub_path, OUT_ISDIR=False, sub_extension='.srt', glue=False):
    file_count = len(video_list)
    done = 0
    for video_path in video_list:
        result = model.transcribe(video_path)
        if OUT_ISDIR:
            #TODO Just get filename w/o absolute path and create sub_file that way
            sub_file = os.path.splitext(video_path)[0] + sub_extension
            sub_path = os.path.join(sub_path, sub_file)
        else:
            sub_file = sub_path
            
        if os.path.splitext(sub_file)[1] == '.srt':
            result_to_srt(result, sub_path, glue=glue)
            done += 1
            print(f"{done}/{file_count}")
        else:
            #TODO implement other subtitle formats
            print(f"Unable to create {sub_extension} files. Use '.srt' instead.")
            break